# OS Level Analysis, Sanity check & File inspection, 

## Size & Stats

In [54]:
!(echo -e "File\tDt_mod\tRows\tSize"; \
 for file in ../app/data/*.csv; do\
    fname=$(basename "$file");\
    dt_mod=$(stat -c '%y' "$file" | cut -d' ' -f1 | tr ' ' '_'); \
    rows=$(wc -l < "$file"); \
    size=$(du -h "$file" | awk '{print $1}');\
    echo -e "$fname\t$dt_mod\t$rows\t$size"; \
  done) | column -t


File                                Dt_mod      Rows      Size
jgp.csv                             2025-02-10  538       36K
nfz_hospitalizations_2019-2021.csv  2025-02-10  17685698  1.9G
nfz_hospitalizations_2022.csv       2025-02-10  3508652   326M


## Raw CSV Header

In [56]:
!for file in ../app/data/*.csv; do \
  fname=$(basename "$file"); \
  echo "$fname"; \
  head -n 3 "$file"; \
  echo "";\
done

jgp.csv
group_code|product_code|name
A01|5.51.01.0001001|Zabiegi wewnątrzczaszkowe z powodu poważnego urazu,
A02|5.51.01.0001002|Zabiegi wewnątrzczaszkowe z powodu urazu

nfz_hospitalizations_2019-2021.csv
2017  ;5   ;"10";"2910050705";"03.4500.030.02";"5.51.01.0006100 ";6   ;1   ;1  ;"18-44";"1-2 dni";"<5"
2017  ;9   ;"15";"7772754458";"03.4570.930.02";"5.51.01.0001011 ";3   ;1   ;2  ;"65 i więcej";"6 i więcej dni";"<5"
2017  ;5   ;"06";"6771694570";"03.4100.030.02";"5.51.01.0005057 ";6   ;1   ;1  ;"45-64";"3-5 dni";"<5"

nfz_hospitalizations_2022.csv
ROK;MIESIAC;OW_NFZ;NIP_PODMIOTU;KOD_PRODUKTU_KONTRAKTOWEGO;KOD_PRODUKTU_JEDNOSTKOWEGO;KOD_TRYBU_PRZYJECIA;KOD_TRYBU_WYPISU;PLEC_PACJENTA;GRUPA_WIEKOWA_PACJENTA;PRZEDZIAL_DLUGOSCI_TRWANIA_HOSPITALIZACJI;LICZBA_HOSPITALIZACJI
2022;4;"07";"1132866688";"03.4580.991.02";"5.51.01.0008013";6;2;"K";"65 i więcej";"6 i więcej dni";"<5"
2022;8;"02";"5562239217";"03.4220.030.02";"5.51.01.0001087";3;2;"K";"45-64";"6 i więcej dni";"<5"



## Columns

**Separators**

In [15]:
!(echo -e "File\tColumns"; \
for file in ../app/data/*.csv; do \
    printf "%s\t" "$(basename "$file")"; \
    awk -F';' 'NR==1 {print NF; exit}' "$file"; \
done) | awk -F'\t' '$2 > 1' | column -t


File                                Columns
nfz_hospitalizations_2019-2021.csv  12
nfz_hospitalizations_2022.csv       12


In [54]:
!(echo -e "File\tColumns"; \
for file in ../app/data/*.csv; do \
    printf "%s\t" "$(basename "$file")"; \
    awk -F'|' 'NR==1 {print NF; exit}' "$file"; \
done) | column -t

File                                Columns
jgp.csv                             3
nfz_hospitalizations_2019-2021.csv  1
nfz_hospitalizations_2022.csv       1


In [5]:
import pandas as pd

pd.read_csv("../app/data/nfz_hospitalizations_2019-2021.csv", nrows=1)

,"2017 ;5 ;""10"";""2910050705"";""03.4500.030.02"";""5.51.01.0006100 "";6 ;1 ;1 ;""18-44"";""1-2 dni"";""<5"""
0,"2017 ;9 ;""15"";""7772754458"";""03.4570.930.02""..."


In [ ]:
import glob
import duckdb as db

test = db.query("SELECT * FROM read_csv_auto ('../app/data/nfz_hospitalizations_2019-2021.csv')")
test



┌──────────┬──────────┬──────────┬────────────┬────────────────┬──────────────────┬──────────┬──────────┬──────────┬─────────────┬────────────────┬──────────┐
│ column00 │ column01 │ column02 │  column03  │    column04    │     column05     │ column06 │ column07 │ column08 │  column09   │    column10    │ column11 │
│  int64   │  int64   │ varchar  │   int64    │    varchar     │     varchar      │  int64   │  int64   │  int64   │   varchar   │    varchar     │ varchar  │
├──────────┼──────────┼──────────┼────────────┼────────────────┼──────────────────┼──────────┼──────────┼──────────┼─────────────┼────────────────┼──────────┤
│     2017 │        5 │ 10       │ 2910050705 │ 03.4500.030.02 │ 5.51.01.0006100  │        6 │        1 │        1 │ 18-44       │ 1-2 dni        │ <5       │
│     2017 │        9 │ 15       │ 7772754458 │ 03.4570.930.02 │ 5.51.01.0001011  │        3 │        1 │        2 │ 65 i więcej │ 6 i więcej dni │ <5       │
│     2017 │        5 │ 06       │ 6771694570 